# Statistical Tests

Name: 

Date: 

## Hypothesis Tests

### Means
1. Identify 5 theologically salient lemmas from the top 20 lemmas (end of PS1) and compute per verse rates by book.
2. Are the per-verse rates between Synoptic Gospels and John the same for all 5 lemmas.
3. Report effect sizes using Cohen's d along with confidence intervals.
4. Correct these results for multiple tests using the Bonferroni correction.
5. Correct these results for multiple tests using the Benjamini-Hochberg correction.
6. Why are the numbers from the Bonferroni correction different from the Benjamini-Hochberg correction? Which results should you use and why?

### Proportion
7. Are the proportion of term presence vs term absence in verse the same between Synoptic Gospels and John for all 5 lemmas?
8. Report effect sizes using Cohen's d along with confidence intervals.
9. Correct these results for multiple tests using the Bonferroni correction.
10. Correct these results for multiple tests using the Benjamini-Hochberg correction.
6. Why are the numbers from the Bonferroni correction different from the Benjamini-Hochberg correction? Which results should you use and why?

### ANOVA
12. Do a term rate ANOVA comparison on a theologically salient lemma from the top 20 lemmas across the following groups: {Matthew and Mark, Luke and Acts, Johannine books, Pauline books, all other books}
13. Perform a Post-hoc Tukey HSD to see which pairs differ

### Chi-square Test
14. Build a 2xk contingency table for collocation: presence of target term vs top-k companion terms within verse.
15. Run Chi-square tests of independence; compute standardized residuals to find surprising co-occurrences
16. Visualize a heatmap of residuals
17. Explain results

18. Repeat the ANOVA and Chi-square test sections for 2 other terms.

## Power Analysis Extension

18. Conduct a power analysis for the question in #7 to determine how many verses are needed to detect the specified effect.
13. Simulate the answer.
    <ol type="a">
    <li>Simulate n verses with the proportions found in #8 for Synoptic Gospels and John. Do a test to see if significantly different and save this result.</li>
    <li>Repeat step a many times.</li>
    <li>Calculate power: power = (number of significant findings) / (total simulations)</li>
    <li>Repeat steps a-c for a different n.</li>
    <li>Plot power as a function of number of verses.</li>
    <li>At what number of verses is the power at least 80%? i.e. This is the number of verses you need per text to have an 80% chance of detecting the effect if it's really there.</li>
    </ol>

## Sliding Windows Stretch
21. Repeat ANOVA and Chi-square sections on the same three theological terms, but instead of using verse boundaries, use sliding windows (+/- n tokens)
22. How do results of verse boundaries compare to using sliding windows?